## Cálculo de Flash

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Os vasos flash são equipamentos destinados à separação de componentes de acordo com sua volatilidade. Porém, de forma distinta de uma destilação clássica, o vaso flash possui apenas um estágio de equilíbrio.

Para determinar a modelagem do vaso Flash, o que significa a vazão de vapor e líquido de saída, é necessário conhecer o quanto um componente tende a ser volátil, que é dada pela constante $k_{i}$

Considerando uma solução ideal e um gás ideal, conforme a lei de Raoult: <p>
$$P^{sat}_{i}x_{i} = Py_{i}$$</p> <p>Logo:</p>
<p>$$k_{i} = \frac{y_{i}}{x_{i}}$$ </p>
<p>$$k_{i} = \frac{P^{sat}_{i}}{P}$$ </p>

A pressão de saturação é obtida pela equação de Antoine, cujos parâmetros estão presentes na planilha "tabela_antoine.xlsx"

O cálculo de flash nesse modelo é apenas um balanço de massa, mas que recai num processo iterativo

O balanço é:
<p>$$F = L + V$$</p>
<p>$$Fz_{i} = Vy_{i} + Lx_{i}$$</p>

Que recai na equação de Rachford, após um simples manejo algébrico desse sistema, considerando a constante $k_{i}$

$$ 1 - \sum_{j=0}^{n} \frac{Fz_{i}k_{i}}{F + V(k_{i} - 1)} = 0$$

Essa equação deve ser resolvida em função da vazão de vapor, que sendo uma mistura com três ou mais componentes, exigirá um procedimento iterativo para a resolução, como o método de Newton-Raphson.
<p>Outra observação relevante é que a mistura que será separada deverá estar em estado de equilíbrio. Dessa forma, deverá estar entre o ponto de bolha e de orvalho. Do contrário, sua resolução é impossível.</p>

In [5]:
def Antoine(A,B,C,T):
    return 10**(A - B/(T + C))

In [6]:
df = pd.read_excel('tabela_antoine.xlsx')
df.set_index('Compound Name',inplace=True)

In [186]:
df.head()

,Formula,A,B,C,TMIN,TMAX
Compound Name,,,,,,
acetone,C3H6O,7.23160,1277.030,237.23,-32,77
acetic-acid,C2H4O2,7.29960,1479.020,216.82,17,157
acetonitrile,C2H3N,7.07350,1279.200,224.01,-13,117
benzene,C6H6,6.90565,1211.033,220.79,-16,104
butane,C4H10,6.80896,935.860,238.73,-78,19


### Localizando os parâmetros de n-butano, n-pentano, n-hexano e cicloexano para obter a pressão de saturação.

In [97]:
n_butane =  list(df.loc['butane'][1:4].values)
n_heptane =  list(df.loc['heptane'][1:4].values)
n_pentane = list(df.loc['pentane'][1:4].values)
n_hexane = list(df.loc['hexane'][1:4].values)
cyclohexane = list(df.loc['cyclohexane'][1:4].values)

### Parâmetros da alimentação (temperatura, pressão, pressão de saturação dos componentes, fração dos componentes e vazão de alimentação)

In [207]:
T = 110 #temperatura em ºC
P = 3800 #pressão em mmHg
P1SAT =  Antoine(n_butane[0],n_butane[1],n_butane[2],T)
P2SAT = Antoine(n_pentane[0],n_pentane[1],n_pentane[2],T)
P3SAT = Antoine(n_hexane[0],n_hexane[1],n_hexane[2],T)
P4SAT = Antoine(cyclohexane[0],cyclohexane[1],cyclohexane[2],T)
PSAT = [P1SAT,P2SAT,P3SAT,P4SAT]
zi = [0.05,0.5,0.3,0.15]
F = 1

### Modelo da equação do Flash

In [211]:
def flash(zi,PSAT,F,P):
    
    ### cálculo de ki
    def k_i(Psat,P):
        return Psat/P
    
    k = [k_i(PSAT[i],P) for i in range(len(PSAT))] 
    
    ## equação de Rachford
    def Rachford(zi,ki,F,V):
        soma = sum([ (F*zi[i]*ki[i])/(F + V*(k[i] - 1)) for i in range(len(zi)) ])    
        raiz = 1 - soma
        return raiz
    
    ## modelo de diferenças finitas para cálculo de derivada
    def derivada(Rachford,zi,ki,F,V):
        df = (Rachford(zi,ki,F,V+0.0001) - Rachford(zi,ki,F,V-0.0001))/0.0002
        return df
    
    ## cálculo do ponto de bolha
    def Bubble(PSAT,zi):
        bubble_point = sum([PSAT[i]*zi[i] for i in range(len(zi))])
        return bubble_point
    
    ## cálculo do ponto de orvalho
    def Dew(PSAT,zi):
        dew_point = 1/sum([(zi[i]/PSAT[i]) for i in range(len(zi))])
        return dew_point
    
    ## chute inicial
    V0 = 0.5*F
    
    ## método de Newton Raphson para resolução do problema
    def NewtonRaphson(V0, zi, ki, F,tol=1e-6):
        for i in range(200):
            V = V0 - Rachford(zi,ki,F,V0)/derivada(Rachford,zi,ki,F,V0)
            error = abs((V - V0)/V)
            if error<tol:
                break
                return V
            else:
                V0 = V
        return V
    
    ## cálculo da pressão do ponto de bolha e orvalho
    bubble, dew = Bubble(PSAT,zi), Dew(PSAT,zi)
    
    ## se estiver entre o ponto de bolha e orvalho, o cálculo será possível
    if (P>dew) and (P<bubble):
        v = NewtonRaphson(F/2, zi, k, F)
        return v, F - v
    else:
        return "Alimentação fora do estado de equilíbrio"

### Vazão de vapor e líquido nas condições do problema

In [213]:
H = flash(zi,PSAT,F,P)
H

(0.4476810946351367, 0.5523189053648633)